In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    average_precision_score
)


In [2]:
csv_path = "flight_weather_merged.csv"
df = pd.read_csv(csv_path)

print("✅ CSV 로드 완료")
print("행 수:", len(df))
print("컬럼 수:", df.shape[1])
df.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_1840\3674691103.py:2: DtypeWarning: Columns (24,25,26,28,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


✅ CSV 로드 완료
행 수: 2843943
컬럼 수: 52


,Unnamed: 0,출발/도착,공항명,항공사,편명,도착지,일자,계획시간,예상시간,출발시간,...,현상번호(국내식),지면온도(°C),풍속_ms,풍향_deg,시정_m,전운량_okta,1층 운고(FT)_결측여부,운형(운형약어),최저운고(100m ),weather_datetime
0,5729485,출발,인천,홍콩항공,HX623,홍콩,20180101,5.0,2340.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5729484,출발,인천,에미레이트항공,EK323,두바이,20180101,5.0,2350.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5729487,출발,인천,에티하드,EY873,아부다비,20180101,15.0,15.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5729488,출발,인천,타이에어아시아엑스,XJ703,돈므앙(방콕),20180101,20.0,20.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5729486,출발,인천,싱가폴항공,SQ603,싱가포르,20180101,5.0,0.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def make_delay_label(x):
    if pd.isna(x):
        return 0
    if any(k in x for k in ["지연", "결항", "회항"]):
        return 1
    return 0

df["is_delay"] = df["RMK_KOR"].apply(make_delay_label)
df["is_delay"].value_counts()


KeyError: 'RMK_KOR'

In [ ]:
df["flight_datetime"] = pd.to_datetime(df["flight_datetime"])

df["weekday"] = df["flight_datetime"].dt.weekday
df["month"] = df["flight_datetime"].dt.month
df["hour"] = df["flight_datetime"].dt.hour
df["flight_date"] = df["flight_datetime"].dt.date


In [ ]:
num_cols = [
    "적설(cm)",
    "풍속_ms",
    "기온(°C)",
    "전운량_okta",
    "강수량(mm)",
    "STD_clean",
    "hour",
    "1층 운고(FT)",
    "이슬점온도(°C)",
    "습도(%)"
]


In [ ]:
cat_cols = [
    "AIRLINE_KOREAN",
    "BOARDING_KOR",
    "ARRIVED_KOR",
    "LINE",
    "IO",
    "일기현상",
    "1층 운형"
]


In [ ]:
# 실제 존재하는 컬럼만 사용
num_cols = [c for c in num_cols if c in df.columns]
cat_cols = [c for c in cat_cols if c in df.columns]

print("📊 수치형:", num_cols)
print("🏷 범주형:", cat_cols)


In [ ]:
# 수치형 → 중앙값
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# 범주형 → UNKNOWN
for col in cat_cols:
    df[col] = df[col].fillna("UNKNOWN")


In [ ]:
print("🔍 수치형 결측치 개수 (대체 전)")
num_missing = df[num_cols].isna().sum().sort_values(ascending=False)
print(num_missing)

print("\n👉 결측치가 하나라도 있는 수치형 컬럼:")
print(num_missing[num_missing > 0])


In [ ]:
print("🔍 범주형 결측치 개수 (대체 전)")
cat_missing = df[cat_cols].isna().sum().sort_values(ascending=False)
print(cat_missing)

print("\n👉 결측치가 하나라도 있는 범주형 컬럼:")
print(cat_missing[cat_missing > 0])


In [ ]:
df = df.sort_values("flight_date")
split_date = df["flight_date"].quantile(0.8)

train_df = df[df["flight_date"] <= split_date]
test_df  = df[df["flight_date"] > split_date]

print("Train:", len(train_df), "Test:", len(test_df))


In [ ]:
X_train = train_df[num_cols + cat_cols]
y_train = train_df["is_delay"]

X_test  = test_df[num_cols + cat_cols]
y_test  = test_df["is_delay"]


In [ ]:
class_weight = {0: 1, 1: 70}

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols)
    ]
)

model = Pipeline([
    ("prep", preprocessor),
    ("clf", LogisticRegression(
        max_iter=3000,
        class_weight=class_weight,
        n_jobs=-1
    ))
])


In [ ]:
model.fit(X_train, y_train)
print("✅ 경량 모델 학습 완료")


In [ ]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("PR-AUC :", average_precision_score(y_test, y_prob))


In [ ]:
for t in [0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f"\n===== Threshold = {t} =====")
    y_pred_t = (y_prob >= t).astype(int)
    print(classification_report(y_test, y_pred_t))
